## Callbacks
학습 중간에 event를 수행
- running rate를 줄일수 있음
- 텐서보드에 레코드를 기록
- tf.record나 체크포인트(model의 weight)를 저장

이번 시간에는 callbacks중 tensorboard 저장하는 방법

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Data Preprocess

In [5]:
os.listdir('../dataset/cifar/')

['labels.txt', 'test', 'train']

In [6]:
train_paths = glob('../dataset/cifar/train/*.png')[:1000]  # 보기용으로 데이터 갯수를 1000개로 제한
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [7]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [8]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [9]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [10]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [11]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [12]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [14]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Callbacks
- Callbacks를 실행할때 로그를 저장할 디렉토리 지정합니다.
- tensorboard 기본 포트 6066

In [15]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))
logdir

'logs\\20191205-213251'

In [16]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/image_summaries.ipynb#scrollTo=IJNpyVyxbVtT

In [ ]:
%tensorboard --logdir logs --port 8808

## Training

http://localhost:6006, http://localhost:8008

In [18]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Epoch 1/10
31/31 [==============================] - 7s 224ms/step - loss: 2.3088 - accuracy: 0.0992 - val_loss: 2.2955 - val_accuracy: 0.1139
Epoch 2/10
31/31 [==============================] - 7s 226ms/step - loss: 2.2952 - accuracy: 0.1322 - val_loss: 2.2977 - val_accuracy: 0.1018
Epoch 3/10
31/31 [==============================] - 9s 280ms/step - loss: 2.2658 - accuracy: 0.1519 - val_loss: 2.1844 - val_accuracy: 0.1986
Epoch 4/10
31/31 [==============================] - 7s 233ms/step - loss: 2.1685 - accuracy: 0.1952 - val_loss: 2.1188 - val_accuracy: 0.2319
Epoch 5/10
31/31 [==============================] - 7s 234ms/step - loss: 2.1095 - accuracy: 0.2066 - val_loss: 2.0678 - val_accuracy: 0.2369
Epoch 6/10
31/31 [==============================] - 7s 234ms/step - loss: 2.0392 - accuracy: 0.2324 - val_loss: 1.9751 - val_accuracy: 0.2802
Epoch 7/10
31/31 [==============================] - 7s 232ms/step - loss: 1.9888 - accuracy: 0.2696 - val_loss: 1.9547 - val_accuracy: 0.2631
Epoch 

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

## LambdaCallback
- tensorboard 안 커스텀한 이미지나 그래프를 넣는 기능
- 여기서는 confuse matrix를 넣는다

In [21]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [22]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [23]:
# Define the per-epoch callback.
# callbacks를 언제 실행할지 정해준다.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
30/31 [============================>.] - ETA: 0s - loss: 1.8079 - accuracy: 0.3066

C:\Users\ynebu\Anaconda3\envs\tensoflow2.0\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 10s 320ms/step - loss: 1.8123 - accuracy: 0.3068 - val_loss: 1.8525 - val_accuracy: 0.3065
Epoch 2/10
30/31 [============================>.] - ETA: 0s - loss: 1.7933 - accuracy: 0.3120

C:\Users\ynebu\Anaconda3\envs\tensoflow2.0\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 10s 331ms/step - loss: 1.8036 - accuracy: 0.3089 - val_loss: 1.7896 - val_accuracy: 0.3387
Epoch 3/10
30/31 [============================>.] - ETA: 0s - loss: 1.7682 - accuracy: 0.3429

...

31/31 [==============================] - 9s 293ms/step - loss: 1.6172 - accuracy: 0.4122 - val_loss: 1.7725 - val_accuracy: 0.3780
Epoch 9/10
30/31 [============================>.] - ETA: 0s - loss: 1.6916 - accuracy: 0.3729 ETA: 3s - los

C:\Users\ynebu\Anaconda3\envs\tensoflow2.0\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 9s 304ms/step - loss: 1.6872 - accuracy: 0.3750 - val_loss: 1.7036 - val_accuracy: 0.3740
Epoch 10/10
30/31 [============================>.] - ETA: 0s - loss: 1.5593 - accuracy: 0.4092

C:\Users\ynebu\Anaconda3\envs\tensoflow2.0\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 9s 289ms/step - loss: 1.5601 - accuracy: 0.4081 - val_loss: 1.6495 - val_accuracy: 0.4052


## Expert

In [24]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [25]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [26]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [27]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [28]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [29]:
# 언제 tensorboard에 넣을거냐?
# 매 step마다 넣어주면 속도가 느려진다. epoch마다 하는게 적당하다.
file_writer = tf.summary.create_file_writer(logdir)

In [ ]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        # 위에  마지막 이미지를 넣어줌
        # step은 epoch을 적용해줘야함.
        tf.summary.image('input_image', images, step=step + (step * epoch))
        tf.summary.scalar('train_loss', train_loss.result(), , step=step + (step * epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.result(), , step=step + (step * epoch))

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

## Tensorboard 띄우는 법
- command 창에서 logs 폴더로 들어감
- tensorboard --logdir=./
- 웹브라우져에서 localhost:6006